In [76]:
import math
import scipy
import numpy as np
from scipy.stats import norm 
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [77]:
""" Problem 1 """

def Newton_Cotes(g, a, b, N, method):
    interval = (b - a)/N
    if method == "midpoint":
        x = np.linspace(a + interval / 2 , b - interval / 2, N)
        summation = g(x).sum()
        return interval * summation
    elif method == "trapezoid":
        x = np.linspace(a + interval, b - interval, N-1)
        summation = g(a) + g(b) + 2 * g(x).sum()
        return interval /2 * summation
    elif method == "Simpsons":
        x= np.linspace(a + interval / 2 , b - interval / 2, 2*N-1)
        summation = g(a) + g(b) + 4* g(x[::2]).sum() + 2* g(x[1::2]).sum()
        return interval / 6 * summation
    
g = lambda x: 0.1* (x**4) -1.5 * (x**3) + 0.53 * (x**2) + 2 * x + 1
print("Midpoint Error:", Newton_Cotes(g, -10, 10, 100, "midpoint") - 4373 - 1/3)
print("Trapezoid Error:", Newton_Cotes(g, -10, 10, 100, "trapezoid") - 4373 - 1/3)
print("Simpsons Error:", Newton_Cotes(g, -10, 10, 100, "Simpsons") - 4373 - 1/3)

Midpoint Error: -1.36857333333
Trapezoid Error: 2.73722666667
Simpsons Error: 2.66666672057e-05


In [78]:
""" Problem 2 """

def norm_aprox(mu, sigma, N, k):
    zvec = np.linspace(mu - sigma * k, mu + sigma * k, N)
    wvec = np.zeros(N)
    for i in range(len(wvec)):
        if i == 0:
            wvec[i] = norm.cdf((zvec[0]+zvec[1])/2, mu, sigma)
        elif i == N-1:
            wvec[i] =  1 - norm.cdf((zvec[N-2] + zvec[N-1])/2, mu, sigma)
        else:
            zmin = (zvec[i-1] + zvec[i]) / 2
            zmax = (zvec[i] + zvec[i+1]) / 2
            wvec[i] = norm.cdf(zmax, mu, sigma) - norm.cdf(zmin, mu, sigma)
    return zvec, wvec

zvec, wvec = norm_aprox(0, 1, 11, 3)
print(zvec)
print(wvec)

[-3.  -2.4 -1.8 -1.2 -0.6  0.   0.6  1.2  1.8  2.4  3. ]
[ 0.00346697  0.01439745  0.04894278  0.11725292  0.19802845  0.23582284
  0.19802845  0.11725292  0.04894278  0.01439745  0.00346697]


In [79]:
""" Problem 3 """

def lognorm_aprox(mu, sigma, N, k):
    zvec, wvec = norm_aprox(mu, sigma, N, k)
    return np.exp(zvec), wvec

zvec, wvec = lognorm_aprox(0, 1, 11, 3)
print(zvec)
print(wvec)

[  0.04978707   0.09071795   0.16529889   0.30119421   0.54881164   1.
   1.8221188    3.32011692   6.04964746  11.02317638  20.08553692]
[ 0.00346697  0.01439745  0.04894278  0.11725292  0.19802845  0.23582284
  0.19802845  0.11725292  0.04894278  0.01439745  0.00346697]


In [80]:
""" Problem 4 """
zvec, wvec = lognorm_aprox(10.5, 0.8, 11, 3)
print(zvec)
print(wvec)

print("Aprox Expectation", (zvec@wvec).sum())
print("Model Expectation", np.exp(10.5 + 0.8 ** 2 / 2))

[   3294.46807528    5324.10552531    8604.15065402   13904.94762458
   22471.42991992   36315.50267425   58688.55427462   94845.07026492
  153276.69022932  247706.53553245  400312.19132988]
[ 0.00346697  0.01439745  0.04894278  0.11725292  0.19802845  0.23582284
  0.19802845  0.11725292  0.04894278  0.01439745  0.00346697]
Aprox Expectation 50352.4561928
Model Expectation 50011.0870085


In [83]:
""" Problem 5 """

def error(vec, a = -10, b = 10):
    system = np.zeros(6)
    for i in range(len(vec)):
        system[i] = (b**(i+1) - a**(i+1))/(i+1) - \
        (vec[0]* (vec[3]** i) + vec[1]* (vec[4]**i) + vec[2]* (vec[5]**i))
    return system

x0 = np.ones(6)
result = opt.root(error, x0, tol=1e-15).x
wvec = result[:3]
xvec = result[3:]
g = lambda x: 0.1* (x**4) -1.5 * (x**3) + 0.53 * (x**2) + 2 * x + 1
print("Quadrature Error:", wvec@g(xvec) - 4373 - 1/3)

Quadrature Error: -3.03146396874e-13


In [84]:
""" Problem 6 """
integral = scipy.integrate.quad(g, -10, 10)[0]
print("Scipy Error:", integral - 4373 - 1/3)

Scipy Error: 6.063483048990292e-13


In [75]:
""" Problem 7 """

def monte_carlo(g, Omega, N):
    integral = 0
    for i in range(N):
        val1 = np.random.uniform(Omega[0,0], Omega[0,1])
        val2 = np.random.uniform(Omega[1,0], Omega[1,1])
        integral += g([val1,val2])
    return 4 / N * integral

N = 1000
Omega = np.array([[-1,1],[-1,1]])
g = lambda x: 1 if x[0] ** 2 + x[1] ** 2 <= 1 else 0
print(monte_carlo(g, Omega, N))

def smallest_N(g, Omega, tol = 5*1e-4):
    N = 100
    aprox = 3.14
    while abs(3.1415 - aprox) > tol:
        aprox = monte_carlo(g, Omega, N)
        N += 1
    return N

print(smallest_N(g, Omega))

3.168
341


In [85]:
""" Problem 8 """

#prime number generator from stackexchange
def n_prime(N):
    primes  = []
    number = 2
    while len(primes) < N:
        ptest    = [number for i in primes if number%i == 0]
        primes  += [] if ptest else [number]
        number += 1
    return primes

def quasi_monte_carlo(n, s, method):

    primes = np.array(n_prime(n))
    
    if method == "Weyl":
        seq = n * primes**(1/2)
        seq = seq[:s]
        return seq % 1
    
    if method == "Haber":
        seq = n* (n+1) / 2 * primes**0.5
        seq = seq[:s]
        return seq % 1
        
    if method == "Niederreiter":
        seq = np.zeros((s, n))
        for i in range(1, n + 1):
            for j in range(1, s + 1):
                seq[j - 1, i - 1] = ((i) * 2. ** ((j) / (j + 1.)) - math.floor((i) * 2. ** ((j) / (j + 1.))))
        seq = seq[::2,::4].reshape(1,4)
        return seq % 1

    if method == "Baker":
        seq = np.zeros((s,n))
        for i in range(1, n+1):
            for j in range(1, s+1):
                seq[j-1, i-1] = ((i+1)*np.exp(j + 1)) - math.floor((i + 1) * np.exp(j + 1))
        seq = seq[::2, ::4].reshape(1,4)
        return seq % 1

In [ ]:
""" Problem 9 """

def pi_qmc(g, Omega, N = 1000, method = "Weyl"):
    integral = 0
    for i in range(2, N):
        val1 = quasi_monte_carlo(i, 2, method)[2]
        val2 = quasi_monte_carlo(i, 2, method)[3]
        #don't understand how to scale the integral 
        #or how to truly apply montecarlo since
        #quasi MC returns 4 values not 2
        integral += g([val1,val2])
    return 4 / N * integral

Omega = np.array([[-1,1],[-1,1]])
g = lambda x: 1 if x[0] ** 2 + x[1] ** 2 <= 1 else 0
pi_qmc(g, Omega)